Load Variables

In [2]:
%run "scripts/load_daily_variables.py"

Loading Variables: 100%|██████████| 26/26 [02:55<00:00,  6.73s/it]  


High Beta with Recent Elevated Volatility and Activity

In [4]:
hbeta_vol_regime_5day = [sym for sym in symbols if 
                    (sym in results_finvizsearch.loc[results_finvizsearch['Beta'] > 2]['Ticker'].to_list()) and 
                    (symbols[sym].ATRs_Traded.replace(np.inf, np.nan).dropna().iloc[-5:] >= 1.5).any() and
                    (symbols[sym].RVol.replace(np.inf, np.nan).dropna().iloc[-5:] >= 1.5).any()
                    ]
hbeta_vol_regime_20day = [sym for sym in symbols if 
                    (sym in results_finvizsearch.loc[results_finvizsearch['Beta'] > 2]['Ticker'].to_list()) and 
                    (symbols[sym].ATRs_Traded.replace(np.inf, np.nan).dropna().iloc[-20:] >= 1.5).any() and
                    (symbols[sym].RVol.replace(np.inf, np.nan).dropna().iloc[-20:] >= 1.5).any() and
                    (sym not in hbeta_vol_regime_5day)
                    ]

In [6]:
mask = daily_quant_rating_df.index.isin(hbeta_vol_regime_5day)
col = daily_quant_rating_df.columns[-2]
(
daily_quant_rating_df.loc[mask, col]
.sort_values(ascending=False)
.head(50)
)

Symbol
LASR    4.977545
GLXY    4.883234
TPC     4.859281
EBS     4.847305
APP     4.806886
FLNC    4.738024
NVTS    4.643713
DAVE    4.550898
BFLY    4.497297
SFIX    4.427027
TXG     4.375676
CDNA    4.181081
PACB    3.959459
VIK     3.789189
APLD    3.490718
COMP    3.490099
LMND    3.475937
HOOD    3.457374
RDDT    3.457030
DPRO    3.450498
RKT     3.438811
ADPT    3.432623
SMTC    3.407528
TMDX    3.386559
NXT     3.309557
AFRM    3.168615
ACHR    3.154864
SHOP    3.154520
NVAX    3.122894
U       3.087487
WRBY    3.030423
UMAC    3.028017
PTON    3.022516
UCTT    3.017704
OPEN    3.014851
TREE    2.970952
AEHR    2.963046
ODD     2.938639
OCGN    2.925232
LIND    2.890168
MSTR    2.887075
AEVA    2.863011
BEAM    2.855792
TBCH    2.820041
PDYN    2.789103
KC      2.778790
JOBY    2.772946
BZH     2.723444
PCT     2.713475
RDW     2.710382
Name: 2026-01-15, dtype: float64

In [7]:
mask = daily_quant_rating_df.index.isin(hbeta_vol_regime_20day)
col = daily_quant_rating_df.columns[-2]
(
daily_quant_rating_df.loc[mask, col]
.sort_values(ascending=False)
.head(50)
)

Symbol
GRAL    4.979042
CRDO    4.970060
HUT     4.907186
CTRN    4.868263
BTSG    4.754491
CCL     4.601796
GAP     4.598802
AS      4.595808
CUK     4.501497
OS      4.440541
PSIX    4.227027
DAN     3.948649
INGM    3.875676
LSPD    3.805405
FDMT    3.764865
NCLH    3.659459
PRAX    3.497250
SKYT    3.496906
SLS     3.494156
ONDS    3.493469
REAL    3.492437
BE      3.491750
RKLB    3.491062
SYRE    3.451530
SOFI    3.445686
JMIA    3.430560
LC      3.428498
OMER    3.402372
AMPX    3.386903
VRT     3.385184
CTMX    3.384496
ASTS    3.367996
CENX    3.365246
NBIS    3.342214
ADEA    3.340495
Z       3.301980
ZG      3.282178
INOD    3.227054
SONO    3.221898
CVNA    3.182365
IMNM    3.180646
BKSY    3.168958
RIOT    3.142833
AMSC    3.117738
NAGE    3.111207
LIF     3.104331
HTZ     3.069612
RUN     3.060674
SAIA    3.053455
AUR     3.043142
Name: 2026-01-15, dtype: float64

YTD Performance vs. Quant Rating

In [8]:
import plotly.express as px
quant_col = daily_quant_rating_df.columns[-2]
quant_rating = daily_quant_rating_df[quant_col].reset_index().rename(columns={quant_col:'Quant Rating'})
perf = results_finvizsearch[['Ticker', 'Performance (YearToDate)']]
df = quant_rating.merge(perf, left_on='Symbol', right_on='Ticker', how='inner')
# df = df.loc[df.Ticker.isin(fu.sector_industry_member_search('Basic Materials', level='sector'))]
# Assign color based on logic:
# quant rating > 4.8 & Performance < 50 -> green
# quant rating < 3 & Performance > 50 -> red
# else -> gray
def assign_color(row):
    if (row['Quant Rating'] > 4.8) and (row['Performance (YearToDate)'] < 50):
        return 'green'
    elif (row['Quant Rating'] < 3) and (row['Performance (YearToDate)'] > 50):
        return 'red'
    elif (row['Quant Rating'] > 2.5) and (row['Quant Rating'] < 4) and (row['Performance (YearToDate)'] > 50):
        return 'yellow'
    else:
        return 'gray'

df['Color'] = df.apply(assign_color, axis=1)

px.scatter(
    df.assign(
        Color_Label=df['Color'].map({'gray': 'uncertain', 'green': 'undervalued', 'red': 'overvalued', 'yellow': 'worth watching'})
    ),
    x='Performance (YearToDate)',
    y='Quant Rating',
    hover_data=['Ticker'],
    color='Color_Label',
    color_discrete_map={'uncertain': 'gray', 'undervalued':'green', 'overvalued':'red', 'worth_watching': 'yellow'},
    category_orders={'Color_Label': ['uncertain', 'undervalued', 'overvalued', 'worth_watching']},
    labels={'Color_Label': 'Category'}
)